# Visualize best saved results of run.py

In [ ]:
# Imports.
import sys, os
sys.path.append('../../')

from flex.tools.utils import get_ground, rotmat2aa
from psbody.mesh.colors import name_to_rgb
from flex.tools.config import Config
from omegaconf import OmegaConf
from psbody.mesh import Mesh
import meshplot as mp
import numpy as np
import smplx
import torch
import mano

In [ ]:
# Helper functions and variables.

def get_sbj(cfg, res):
    # ---- (*) BODY
    sbj_m = smplx.create(model_path=f'../../{cfg.smplx_dir}',
                        model_type='smplx',
                        gender='neutral',
                        num_pca_comps=np.array(24),
                        batch_size=1).to('cpu').eval()
    body_params = {'transl': torch.Tensor(res['transl'].reshape(1,3)),
                   'global_orient': torch.Tensor(res['global_orient'].reshape(1,3)),
                   'body_pose': torch.Tensor(res['pose'].reshape(1,63))}
    bm = sbj_m(**body_params)
    body_vertices = bm.vertices.detach().cpu().numpy()[0]
    body_vertices[..., 2] -= body_vertices[..., 2].min(-1, keepdims=True)[0]
    return body_vertices, sbj_m.faces


def get_rh(cfg, res):
    # ---- (*) RHAND
    rh_m = mano.load(model_path=f'../../{cfg.mano_dir}',
                     is_right=True,
                     model_type='mano',
                     gender='neutral',
                     num_pca_comps=45,
                     flat_hand_mean=True,
                     batch_size=1).to('cpu').eval()
    return res['rh_verts'].reshape(-1, 3), rh_m.faces

In [ ]:
# Set example.
obj_name, recept, ornt, index = 'stapler', 'receptacle_aabb_TvStnd1_Top3_frl_apartment_tvstand', 'all', 0

# Load dataset.
recept_dict = dict(np.load('../../data/replicagrasp/receptacles.npz', allow_pickle=1))
dset_info_dict = dict(np.load('../../data/replicagrasp/dset_info.npz', allow_pickle=1))

# Load tto optimization human result.
res = dict(np.load(f'../../save/{obj_name}/{recept}/{ornt}_{index}.npz', allow_pickle=True))

# Setup cfg.
cfg = OmegaConf.structured(Config)
config_yaml = OmegaConf.load('../configs/flex.yaml')
cfg = OmegaConf.merge(cfg, config_yaml)
cfg.obj_name = obj_name

# Mesh path.
mesh_pth = '../../data/obj/contact_meshes/'

In [ ]:
# ---> Load data.
transl_grab, orient_grab, recept_idx = dset_info_dict[f'{obj_name}_{recept}_{ornt}_{index}']
recept_v, recept_f = recept_dict[recept][recept_idx][0], recept_dict[recept][recept_idx][1]

# ---> Visualize receptacle meshes in scene.
mp_viewer = mp.plot(recept_v, recept_f, name_to_rgb['yellow'])

# ---> Visualize ground for context (center it to mean of rigid/articulated object).
grnd_mesh = get_ground(grnd_size=10, offset=0.0)
xmean, ymean, _ = recept_v.mean(0)
grnd_mesh.v[:, 0] += xmean
grnd_mesh.v[:, 1] += ymean
mp_viewer.add_mesh(grnd_mesh.v, grnd_mesh.f, name_to_rgb['brown'])

# ---> Visualize object to be grasped in scene.
object_mesh = Mesh(filename=os.path.join(mesh_pth, obj_name + '.ply'), vscale=1.)
obj_verts = np.matmul(object_mesh.v, orient_grab.T) + transl_grab
mp_viewer.add_mesh(obj_verts, object_mesh.f, name_to_rgb['blue'])

# -----------
# Get top-5 result.
for idx, color in enumerate(['green', 'pink', 'yellow', 'gray', 'orange']):
    res_i = dict(res['arr_0'].item())['final_results'][idx]
    res_i_new = res_i.copy()
    res_i_new['global_orient'] = rotmat2aa(torch.Tensor(res_i['global_orient']).reshape(1,1,1,9)).reshape(1, 3).numpy()
    # Visualize human.
    sbj_v, sbj_f = get_sbj(cfg, res_i_new)
    mp_viewer.add_mesh(sbj_v, sbj_f, name_to_rgb[color])
    # (Optional) Uncomment below lines to visualize target rhand grasp.
    # rh_v, rh_f = get_rh(cfg, res_i_new)
    # mp_viewer.add_mesh(rh_v, rh_f, name_to_rgb['red'])

print("Done!")